<a href="https://colab.research.google.com/github/heejeongJ/DL_pt/blob/Convolutional-Neural-Network_CNN/Convolutional_Neural_Network_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

import torchvision.datasets as dset
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt

print(torch.__version__)

2.1.0+cu118


In [19]:
batch_size = 256
learning_rate = 0.0002
num_epoch = 10

In [20]:
mnist_train = dset.MNIST(root="../", train=True, transform = transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST(root="../", train=True, transform = transforms.ToTensor(), target_transform=None, download=True)

In [21]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
print(mnist_test.__getitem__(0)[0].size(), mnist_test.__len__())

print(len(mnist_train), len(mnist_test))


torch.Size([1, 28, 28]) 60000
torch.Size([1, 28, 28]) 60000
60000 60000


In [22]:
train_loader = DataLoader(mnist_train, batch_size = batch_size, shuffle = True, num_workers=2, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size = batch_size, shuffle = False, num_workers=2, drop_last=True)

In [27]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(in_channels = 1, out_channels=16, kernel_size=5),
        nn.ReLU(),
        nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size=5),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=5),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(64*3*3, 100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )

  def forward(self,x):
      out = self.layer(x)
      out= out.view(batch_size, -1)

      out = self.fc_layer(out)
      return out


In [28]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = CNN().to(device)

loss_func = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

cuda:0


In [29]:
loss_arr = []
for i in range(num_epoch):
  for j, [image, label] in enumerate(train_loader) :
    x=image.to(device)
    y_=label.to(device)

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output, y_)
    loss.backward()
    optimizer.step()

    if j%1000 == 0:
      print(loss)
      loss_arr.append(loss.cpu().detach().numpy())

tensor(2.2995, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2383, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1007, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0923, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0316, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0904, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0211, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0803, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0231, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0169, device='cuda:0', grad_fn=<NllLossBackward0>)


In [30]:
correct = 0
total = 0

with torch.no_grad():
  for image, label in test_loader:

    x=image.to(device)
    y_=label.to(device)

    output = model.forward(x)

    _,output_index = torch.max(output, 1)

    total += label.size(0)

    correct += (output_index == y_).sum().float()

  print("Accuracy of Test Data: {}%".format(100*correct/total))

Accuracy of Test Data: 99.013427734375%
